In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split,cross_val_score

In [2]:
df = pd.read_csv('finalChildMalnutrition.csv', index_col = 0)

In [3]:
df = df.fillna(df[['fever','Mother_BMI','diar','Father_Edu','currently_working_mot']].mean())

In [4]:
df.tail(10)

,Division,Age at 1st Birth,Child Age,stunting,underweight,wasting,diar,fever,ari,Mother_BMI,Birth_Order,Mother_edu,wealth_index_cat,Father_Edu,residence,sex,currently_working_mot,Breastfeeding,household_no
6955,7,22,40,0,0,0,0.000000,0.000000,0,21.694767,2,1,1,0.000000,0,0,0.000000,0,5
6956,7,22,55,0,0,0,0.000000,0.000000,0,21.694767,1,1,1,0.000000,0,0,0.000000,0,5
6957,7,19,31,1,0,0,0.000000,1.000000,0,21.694767,3,1,1,0.000000,0,1,0.000000,0,6
6958,7,19,59,1,1,0,0.000000,0.369215,0,21.694767,2,1,1,0.000000,0,1,0.000000,0,6
6959,7,17,42,0,0,0,0.048426,0.369215,0,21.694767,3,0,1,1.000000,0,1,0.000000,0,17
6960,7,17,58,0,0,1,0.048426,0.369215,0,21.694767,2,0,1,0.000000,0,0,0.000000,0,17
6961,7,18,22,1,0,0,0.048426,0.369215,0,21.694767,1,1,1,0.000000,0,0,0.000000,1,17
6962,7,22,49,0,0,0,0.048426,0.369215,0,21.694767,1,1,1,0.000000,0,0,0.000000,0,3
6963,7,18,53,1,1,0,0.048426,0.369215,0,21.694767,3,0,1,0.449088,0,0,0.000000,0,6
6964,7,18,32,1,1,0,0.048426,0.369215,0,21.694767,1,0,1,0.449088,0,0,0.250862,0,6


In [5]:
# wasting
# -----------

print(df['wasting'].value_counts())

# underweight
# -----------

print(df['stunting'].value_counts())

# stuting
# -----------
print(df['underweight'].value_counts())

0    5978
1     987
Name: wasting, dtype: int64
0    4450
1    2515
Name: stunting, dtype: int64
0    4727
1    2238
Name: underweight, dtype: int64


In [6]:
X  = df.iloc[:, [1,2,6,7,8,9,10,11,12,13,14,15,16,17,18]].values

y_stun = df.iloc[:, 3].values
y_underweight = df.iloc[:, 4].values
y_wast = df.iloc[:, 5].values

In [7]:
# from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE

In [8]:
# rus = SMOTETomek(ratio = 'auto')
smote = SMOTE('minority')

In [9]:
# X, y_wast = rus.fit_sample(X, y_wast)
X_sm, y_wast_sm = smote.fit_sample(X, y_wast)
X_sm1, y_underweight_sm = smote.fit_sample(X, y_underweight)
X_sm2, y_stun_sm = smote.fit_sample(X, y_stun)

In [10]:
print(X_sm.shape)
print(X_sm1.shape)
print(X_sm2.shape)

(11956, 15)
(9454, 15)
(8900, 15)


In [11]:
from sklearn.utils import class_weight

In [12]:
c_weight = class_weight.compute_class_weight('balanced', np.unique(y_wast), y_wast)
c_weight1 = class_weight.compute_class_weight('balanced', np.unique(y_underweight), y_underweight)
c_weight2 = class_weight.compute_class_weight('balanced', np.unique(y_stun), y_stun)

In [13]:
x_train_wast, x_test_wast, y_train_wast, y_test_wast = train_test_split(X_sm, y_wast_sm, test_size = 0.1, random_state = 41)
x_train_under, x_test_under, y_train_under, y_test_under = train_test_split(X_sm1, y_underweight_sm, test_size = 0.1, random_state = 41)
x_train_stun, x_test_stun, y_train_stun, y_test_stun = train_test_split(X_sm2, y_stun_sm, test_size = 0.1, random_state = 41)

In [14]:
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler

sc = StandardScaler()
x_train_stun = sc.fit_transform(x_train_stun)
x_train_underWeight = sc.fit_transform(x_train_under)
x_train_wast = sc.fit_transform(x_train_wast)

x_test_stun = sc.transform(x_test_stun)
x_test_underWeight = sc.transform(x_test_under)
x_test_wast = sc.transform(x_test_wast)

In [15]:
import keras
from keras.models import Sequential              # initialize the neural network
from keras.layers import Dense, Dropout  

Using TensorFlow backend.


In [16]:
classifiers = Sequential()

In [17]:
# 1st hidden layer & input layer

classifiers.add(Dense(8, input_shape=(15,), activation='relu',kernel_initializer='uniform')) 
# classifiers.add(Dropout(rate = 0.2))

Instructions for updating:
Colocations handled automatically by placer.


In [18]:
# 2nd hidden layer

classifiers.add(Dense(8, activation='relu',kernel_initializer='uniform')) 
# classifiers.add(Dropout(rate = 0.2))

In [19]:
# # 3rd hidden layer

# classifiers.add(Dense(8, activation='relu',kernel_initializer='uniform')) 
# classifiers.add(Dropout(rate = 0.2))

In [20]:
# # 4th hidden layer

# classifiers.add(Dense(8, activation='relu',kernel_initializer='uniform')) 
# # classifiers.add(Dropout(rate = 0.2))

In [21]:
# # 5th hidden layer

# classifiers.add(Dense(8, activation='relu',kernel_initializer='uniform')) 
# classifiers.add(Dropout(rate = 0.2))

In [22]:
# # 6th hidden layer

# classifiers.add(Dense(8, activation='relu',kernel_initializer='uniform')) 
# classifiers.add(Dropout(rate = 0.2))

In [19]:
# adding output layer
#--------------------------

classifiers.add(Dense(1, activation='sigmoid',kernel_initializer='uniform'))

In [20]:
# compiling ANN
#----------------

classifiers.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
# train wasting
# ---------------

classifiers.fit(x_train_wast,y_train_wast, batch_size = 3, epochs = 100, class_weight = c_weight)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
10760/10760 [==============================] - 3s 264us/step - loss: 0.6790 - acc: 0.5651
Epoch 2/100
10760/10760 [==============================] - 2s 229us/step - loss: 0.6578 - acc: 0.6152
Epoch 3/100
10760/10760 [==============================] - 3s 236us/step - loss: 0.6313 - acc: 0.6520
Epoch 4/100
10760/10760 [==============================] - 2s 217us/step - loss: 0.5970 - acc: 0.6823
Epoch 5/100
10760/10760 [==============================] - 2s 212us/step - loss: 0.5690 - acc: 0.6960
Epoch 6/100
10760/10760 [==============================] - 2s 215us/step - loss: 0.5500 - acc: 0.7068
Epoch 7/100
10760/10760 [==============================] - 2s 206us/step - loss: 0.5329 - acc: 0.7107
Epoch 8/100
10760/10760 [==============================] - 2s 210us/step - loss: 0.5232 - acc: 0.7225
Epoch 9/100
10760/10760 [==============================] - 2s 202us/step - loss: 0.5157 - acc: 0.7224
Epoch 10/100
10760/10760 [========

10760/10760 [==============================] - 3s 280us/step - loss: 0.3902 - acc: 0.8143
Epoch 79/100
10760/10760 [==============================] - 4s 341us/step - loss: 0.3893 - acc: 0.8166
Epoch 80/100
10760/10760 [==============================] - 3s 236us/step - loss: 0.3888 - acc: 0.8191
Epoch 81/100
10760/10760 [==============================] - 2s 206us/step - loss: 0.3897 - acc: 0.8186
Epoch 82/100
10760/10760 [==============================] - 2s 210us/step - loss: 0.3885 - acc: 0.8191
Epoch 83/100
10760/10760 [==============================] - 2s 204us/step - loss: 0.3864 - acc: 0.8194
Epoch 84/100
10760/10760 [==============================] - 2s 203us/step - loss: 0.3869 - acc: 0.8204
Epoch 85/100
10760/10760 [==============================] - 2s 205us/step - loss: 0.3874 - acc: 0.8179
Epoch 86/100
10760/10760 [==============================] - 2s 201us/step - loss: 0.3875 - acc: 0.8175
Epoch 87/100
10760/10760 [==============================] - 2s 200us/step - loss: 0.38

In [22]:
# prediction wasting
# ------------------

y_pred_wast = classifiers.predict(x_test_wast)
y_pred_wast = (y_pred_wast > 0.5)

In [23]:
# wasting result
# ----------------

print('for test data:\n------------------')
print(round(accuracy_score(y_test_wast,y_pred_wast)*100, 6))
print('\n------------------------------------')
print(confusion_matrix(y_test_wast,y_pred_wast))
print(classification_report(y_test_wast,y_pred_wast))

for test data:
------------------
84.197324

------------------------------------
[[588  25]
 [164 419]]
              precision    recall  f1-score   support

           0       0.78      0.96      0.86       613
           1       0.94      0.72      0.82       583

   micro avg       0.84      0.84      0.84      1196
   macro avg       0.86      0.84      0.84      1196
weighted avg       0.86      0.84      0.84      1196



In [24]:
classifiers1 = Sequential()

In [25]:
# 1st hidden layer & input layer

classifiers1.add(Dense(8, input_shape=(15,), activation='relu',kernel_initializer='uniform')) 
# classifiers1.add(Dropout(rate = 0.2))

In [26]:
# 2nd hidden layer

classifiers1.add(Dense(8, activation='relu',kernel_initializer='uniform')) 
# classifiers1.add(Dropout(rate = 0.2))

In [30]:
# # 3rd hidden layer

# classifiers1.add(Dense(8, activation='relu',kernel_initializer='uniform')) 
# classifiers1.add(Dropout(rate = 0.2))

In [27]:
# adding output layer
#--------------------------

classifiers1.add(Dense(1, activation='sigmoid',kernel_initializer='uniform'))

In [28]:
# compiling ANN
#----------------

classifiers1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [29]:
# train underweight
# ---------------

classifiers1.fit(x_train_under,y_train_under, batch_size = 3, epochs = 100, class_weight = c_weight1)

Epoch 1/100
8508/8508 [==============================] - 2s 274us/step - loss: 0.6743 - acc: 0.5771
Epoch 2/100
8508/8508 [==============================] - 2s 258us/step - loss: 0.6559 - acc: 0.6188
Epoch 3/100
8508/8508 [==============================] - 2s 236us/step - loss: 0.6499 - acc: 0.6256
Epoch 4/100
8508/8508 [==============================] - 2s 219us/step - loss: 0.6462 - acc: 0.6353
Epoch 5/100
8508/8508 [==============================] - 2s 224us/step - loss: 0.6450 - acc: 0.6323
Epoch 6/100
8508/8508 [==============================] - 2s 218us/step - loss: 0.6439 - acc: 0.6378
Epoch 7/100
8508/8508 [==============================] - 2s 221us/step - loss: 0.6430 - acc: 0.6341
Epoch 8/100
8508/8508 [==============================] - 2s 257us/step - loss: 0.6424 - acc: 0.6381
Epoch 9/100
8508/8508 [==============================] - 2s 234us/step - loss: 0.6429 - acc: 0.6378
Epoch 10/100
8508/8508 [==============================] - 2s 227us/step - loss: 0.6414 - acc: 0.6394

8508/8508 [==============================] - 2s 233us/step - loss: 0.6292 - acc: 0.6500
Epoch 82/100
8508/8508 [==============================] - 2s 257us/step - loss: 0.6285 - acc: 0.6547 1s - loss
Epoch 83/100
8508/8508 [==============================] - 2s 246us/step - loss: 0.6285 - acc: 0.6547
Epoch 84/100
8508/8508 [==============================] - 2s 271us/step - loss: 0.6278 - acc: 0.6529
Epoch 85/100
8508/8508 [==============================] - 2s 257us/step - loss: 0.6283 - acc: 0.6507
Epoch 86/100
8508/8508 [==============================] - 3s 300us/step - loss: 0.6282 - acc: 0.6536
Epoch 87/100
8508/8508 [==============================] - 2s 286us/step - loss: 0.6278 - acc: 0.6523
Epoch 88/100
8508/8508 [==============================] - 2s 290us/step - loss: 0.6273 - acc: 0.6523
Epoch 89/100
8508/8508 [==============================] - 2s 253us/step - loss: 0.6274 - acc: 0.6517
Epoch 90/100
8508/8508 [==============================] - 2s 280us/step - loss: 0.6280 - acc: 

In [30]:
# prediction underweight
# ------------------

y_pred_under = classifiers1.predict(x_test_under)
y_pred_under = (y_pred_under > 0.5)

In [31]:
# underweight result
# ----------------

print('for test data:\n------------------')
print(round(accuracy_score(y_test_under,y_pred_under)*100, 6))
print('\n------------------------------------')
print(confusion_matrix(y_test_under,y_pred_under))
print(classification_report(y_test_under,y_pred_under))

for test data:
------------------
62.896406

------------------------------------
[[240 217]
 [134 355]]
              precision    recall  f1-score   support

           0       0.64      0.53      0.58       457
           1       0.62      0.73      0.67       489

   micro avg       0.63      0.63      0.63       946
   macro avg       0.63      0.63      0.62       946
weighted avg       0.63      0.63      0.62       946



In [32]:
classifiers2 = Sequential()

In [33]:
# 1st hidden layer & input layer

classifiers2.add(Dense(8, input_shape=(15,), activation='relu',kernel_initializer='uniform')) 
# classifiers2.add(Dropout(rate = 0.2))

In [34]:
# 2nd hidden layer

classifiers2.add(Dense(8, activation='relu',kernel_initializer='uniform')) 
# classifiers2.add(Dropout(rate = 0.2))

In [45]:
# # 3rd hidden layer

# classifiers2.add(Dense(8, activation='relu',kernel_initializer='uniform')) 
# classifiers2.add(Dropout(rate = 0.2))

In [35]:
# adding output layer
#--------------------------

classifiers2.add(Dense(1, activation='sigmoid',kernel_initializer='uniform'))

In [36]:
# compiling ANN
#----------------

classifiers2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
# train stunting
# ---------------

classifiers2.fit(x_train_stun,y_train_stun, batch_size = 3, epochs = 100, class_weight = c_weight2)

Epoch 1/100
8010/8010 [==============================] - 2s 283us/step - loss: 0.6659 - acc: 0.6055
Epoch 2/100
8010/8010 [==============================] - 2s 253us/step - loss: 0.6519 - acc: 0.6236
Epoch 3/100
8010/8010 [==============================] - 2s 302us/step - loss: 0.6452 - acc: 0.6373
Epoch 4/100
8010/8010 [==============================] - 2s 292us/step - loss: 0.6380 - acc: 0.6486
Epoch 5/100
8010/8010 [==============================] - 2s 292us/step - loss: 0.6333 - acc: 0.6536
Epoch 6/100
8010/8010 [==============================] - 2s 301us/step - loss: 0.6301 - acc: 0.6568
Epoch 7/100
8010/8010 [==============================] - 2s 310us/step - loss: 0.6277 - acc: 0.6554
Epoch 8/100
8010/8010 [==============================] - 2s 279us/step - loss: 0.6258 - acc: 0.6610
Epoch 9/100
8010/8010 [==============================] - 2s 276us/step - loss: 0.6247 - acc: 0.6652
Epoch 10/100
8010/8010 [==============================] - 2s 290us/step - loss: 0.6230 - acc: 0.6697

8010/8010 [==============================] - 2s 232us/step - loss: 0.5667 - acc: 0.6991
Epoch 83/100
8010/8010 [==============================] - 2s 223us/step - loss: 0.5665 - acc: 0.6974
Epoch 84/100
8010/8010 [==============================] - 2s 236us/step - loss: 0.5672 - acc: 0.6981
Epoch 85/100
8010/8010 [==============================] - 2s 230us/step - loss: 0.5663 - acc: 0.6986
Epoch 86/100
8010/8010 [==============================] - 2s 227us/step - loss: 0.5660 - acc: 0.6949
Epoch 87/100
8010/8010 [==============================] - 2s 240us/step - loss: 0.5650 - acc: 0.6946
Epoch 88/100
8010/8010 [==============================] - 2s 234us/step - loss: 0.5657 - acc: 0.6968
Epoch 89/100
8010/8010 [==============================] - 2s 224us/step - loss: 0.5657 - acc: 0.6991
Epoch 90/100
8010/8010 [==============================] - 2s 226us/step - loss: 0.5638 - acc: 0.6998
Epoch 91/100
8010/8010 [==============================] - 2s 225us/step - loss: 0.5645 - acc: 0.6969
Epo

In [38]:
# prediction stunting
# ------------------

y_pred_stun = classifiers2.predict(x_test_stun)
y_pred_stun = (y_pred_stun > 0.5)

In [39]:
# stunting result
# ----------------

print('for test data:\n------------------')
print(round(accuracy_score(y_test_stun,y_pred_stun)*100, 6))
print('\n------------------------------------')
print(confusion_matrix(y_test_stun,y_pred_stun))
print(classification_report(y_test_stun,y_pred_stun))

for test data:
------------------
71.235955

------------------------------------
[[367  83]
 [173 267]]
              precision    recall  f1-score   support

           0       0.68      0.82      0.74       450
           1       0.76      0.61      0.68       440

   micro avg       0.71      0.71      0.71       890
   macro avg       0.72      0.71      0.71       890
weighted avg       0.72      0.71      0.71       890



In [40]:
# predicting stunting, underweight & wasting

In [44]:
age_at_first_bir = input('mother age at first birth (in year): ' )
child_age        = input('child age (in month): ' )
diar             = float(input('diarrhea [yes(1.0), no(0.0)]: '))
fever            = float(input('fever [yes(1.0), no(0.0)]: '))
ari              = int(input('ari [yes(1), no(0)]: '))
mother_BMI       = float(input('mother BMI: '))
birth_order      = int(input('birth order: '))
mother_edu       = int(input('mother education [yes(1), no(0)]: '))
wealth_condition = int(input('wealth condition [rich(1), poor(0)]: '))
Father_Edu       = float(input('Father education [yes(1.0), no(0)]: '))
residence        = int(input('residence [rural(1) , urban(0)]: '))
gender           = int(input('gender[male(0), female(1)]: ')) 
mother_working   = float(input('mother working [yes(1.0), no(0.0)]: '))
breastfeeding    = int(input('breastfeeding [yes(1), no(0)]: '))
household_no     = int(input('no of family members: '))

mother age at first birth (in year): 16
child age (in month): 13
diarrhea [yes(1.0), no(0.0)]: 0.0
fever [yes(1.0), no(0.0)]: 1.0
ari [yes(1), no(0)]: 1
mother BMI: 20.96
birth order: 2
mother education [yes(1), no(0)]: 0
wealth condition [rich(1), poor(0)]: 0
Father education [yes(1.0), no(0)]: 0.0
residence [rural(1) , urban(0)]: 1
gender[male(0), female(1)]: 1
mother working [yes(1.0), no(0.0)]: 0.0
breastfeeding [yes(1), no(0)]: 1
no of family members: 6


In [40]:
stun  = ''
under = ''
was   = '' 

In [69]:
# new_stun = classifiers2.predict(sc.transform(np.array([[age_at_first_bir, child_age, diar, fever, ari, mother_BMI, birth_order, mother_edu, wealth_condition, Father_Edu, residence, gender, mother_working, breastfeeding, household_no]])))
new_stun = classifiers2.predict(sc.transform(np.array([[18, 47, 0.0, 1.0, 0, 19.71, 2, 0, 0, 0.0, 1, 1, 0.0, 0, 5]])))
new_stun = (new_stun > 0.5)

In [70]:
# new_under = classifiers1.predict(sc.transform(np.array([[age_at_first_bir, child_age, diar, fever, ari, mother_BMI, birth_order, mother_edu, wealth_condition, Father_Edu, residence, gender, mother_working, breastfeeding, household_no]])))
new_under = classifiers1.predict(sc.transform(np.array([[18, 47, 0.0, 1.0, 0, 19.71, 2, 0, 0, 0.0, 1, 1, 0.0, 0, 5]])))
new_under = (new_under > 0.5)

In [71]:
# new_was = classifiers.predict(sc.transform(np.array([[age_at_first_bir, child_age, diar, fever, ari, mother_BMI, birth_order, mother_edu, wealth_condition, Father_Edu, residence, gender, mother_working, breastfeeding, household_no]])))
new_was = classifiers.predict(sc.transform(np.array([[18, 47, 0.0, 1.0, 0, 19.71, 2, 0, 0, 0.0, 1, 1, 0.0, 0, 5]])))
new_was = (new_was > 0.5)

In [72]:
for i in new_stun:
    for j in i:
        if j == True:
            stun = 'not stunting'
        else:
            stun = 'stunting'

for i in new_under:
    for j in i:
        if j == True:
            under = 'not underweight'
        else:
            under = 'underweight'

for i in new_was:
    for j in i:
        if j == True:
            was = 'not wasting'
        else:
            was = 'wasting'

In [81]:









print('child is:\n -------------------\n # ', stun, ';\n\n' , '# ',under , ';\n\n' , '# ',was,';')










child is:
 -------------------
 #  not stunting ;

 #  underweight ;

 #  wasting ;


In [ ]:
new_stun

In [ ]:
new_under

In [ ]:
new_was